In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading
import copy
from contextlib import contextmanager

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka._components.helpers import _import_from_string
from fastkafka.server import Server, run_in_process_until_terminate

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

from fastkafka.testing import change_dir
from fastkafka._components.helpers import generate_app_src

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        1, help="Number of FastKafka instances to run"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    async def main_loop(num_workers: int=num_workers, app: str=app):
        logger.info("Entering whatever()")
        try:
            application = _import_from_string(app)
            logger.info(f"{application=}")
            server = Server(app=application, num_workers=num_workers)
            logger.info("Server object created")

            loop = asyncio.get_event_loop()

            HANDLED_SIGNALS = (
                signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
                signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
            )

            def handle_exit(sig: int) -> None:
                global should_exit
                should_exit = True

            for sig in HANDLED_SIGNALS:
                loop.add_signal_handler(sig, handle_exit, sig)

            should_exit = False

            logger.info("Entering waiting loop...")
            with server.run_in_process():
                while not should_exit:
                    await asyncio.sleep(0.2)
                    print(".", end="")
                    
        except Exception as e:
            typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
            raise typer.Exit(1)
                    
#     logger.info("Entering run()")
    asyncio.run(main_loop())
#     logger.info("Exiting run()")

@_app.command(
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
@contextmanager
def generate_app_in_tmp() -> Generator[None, None, None]:
    with TemporaryDirectory() as d:
        src_path = Path(d) / "main.py"
        generate_app_src(src_path)
        print(f"{src_path=}")
        with change_dir(d):
            import_str = f"{src_path.stem}:kafka_app"
            yield import_str

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

In [ ]:
with generate_app_in_tmp() as import_str:
        def target(result=result):
            runner = CliRunner()
            runner.invoke(_app, ["run", import_str])
            
        with run_in_process_until_terminate(target):
            time.sleep(5)
            print("Done sleeping..")

In [ ]:
result = runner.invoke(_app, ["--help"])

In [ ]:
result = runner.invoke(_app, ["generate-docs", "--help"])

In [ ]:
with generate_app_in_tmp() as import_str:
        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0